In [1]:
import intera_interface
import rospy
import json
import tf
from copy import deepcopy
from sawyer import transformations

In [2]:
rospy.init_node("sawyer_example")

In [3]:
effector = "right_hand"
world = "world"

In [4]:
tfl = tf.TransformListener()
tfb = tf.TransformBroadcaster()

In [5]:
rs = intera_interface.RobotEnable(intera_interface.CHECK_VERSION)

In [6]:
rs.enable()

[INFO] [1554567884.586898]: Robot Enabled


In [7]:
limb = intera_interface.Limb('right')

In [6]:
angles = limb.joint_angles()

In [7]:
angles

{'right_j0': 0.00175390625,
 'right_j1': -1.177833984375,
 'right_j2': 0.0013876953125,
 'right_j3': 2.17882421875,
 'right_j4': 0.0031181640625,
 'right_j5': 0.56447265625,
 'right_j6': 3.3151767578125}

In [8]:
limb.move_to_neutral()

In [10]:
head = intera_interface.head.Head()

In [12]:
head.pan()

-3.1976141929626465

In [15]:
head.set_pan(-1.57)

In [22]:
hope_lower_left = limb.joint_angles()

In [17]:
hope_lower_right

{'right_j0': 0.0704453125,
 'right_j1': -3.5299384765625,
 'right_j2': -3.023064453125,
 'right_j3': -2.736953125,
 'right_j4': -1.4090927734375,
 'right_j5': 1.5120576171875,
 'right_j6': 1.0290107421875}

In [19]:
hope_lower_right = limb.joint_angles()

In [40]:
hope_higher_right = limb.joint_angles()

In [30]:
hope_lower_left = limb.joint_angles()

## Read poses

In [8]:
with open("../config/poses.json") as f:
    extremes = json.load(f)

In [13]:
hope_extremes = [extremes['hope']['higher']['left'], extremes['hope']['higher']['right'],
                 extremes['hope']['lower']['left'], extremes['hope']['lower']['right']]
for i in range(1):
    for pose in hope_extremes:
        limb.move_to_joint_positions(pose)
        rospy.sleep(2)

# Convert extreme poses to cartesian

In [42]:
help(tfb.sendTransform)

Help on method sendTransform in module tf.broadcaster:

sendTransform(self, translation, rotation, time, child, parent) method of tf.broadcaster.TransformBroadcaster instance
    :param translation: the translation of the transformtion as a tuple (x, y, z)
    :param rotation: the rotation of the transformation as a tuple (x, y, z, w)
    :param time: the time of the transformation, as a rospy.Time()
    :param child: child frame in tf, string
    :param parent: parent frame in tf, string
    
    Broadcast the transformation from tf frame child to parent on ROS topic ``"/tf"``.



In [59]:
extremes_cartesian = deepcopy(extremes)

for emotion in extremes:
    for vertical in extremes[emotion]:
        for horizontal in extremes[emotion][vertical]:
            limb.move_to_joint_positions(extremes[emotion][vertical][horizontal])
            rospy.sleep(2)
            position, orientation = list(tfl.lookupTransform(world, effector, rospy.Time(0)))
            extremes_cartesian[emotion][vertical][horizontal] = [position, orientation]

In [62]:
# Uniformize
for emotion in extremes:
    for vertical in extremes[emotion]:
        for horizontal in extremes[emotion][vertical]:
            # All orientations uniform
            extremes_cartesian[emotion][vertical][horizontal][1] = extremes_cartesian[emotion]["higher"]["left"][1]
            # All -x uniform
            extremes_cartesian[emotion][vertical][horizontal][0][0] = extremes_cartesian[emotion]["higher"]["right"][0][0]
            
            # Align lwoer/higher on z axis
            if vertical == "lower":
                extremes_cartesian[emotion][vertical][horizontal][0][2] = extremes_cartesian[emotion]["lower"]["right"][0][2]
            elif vertical == "higher":
                extremes_cartesian[emotion][vertical][horizontal][0][2] = extremes_cartesian[emotion]["higher"]["left"][0][2]

            # Align left/right on y axis
            if horizontal == "left":
                extremes_cartesian[emotion][vertical][horizontal][0][1] = extremes_cartesian[emotion]["lower"]["left"][0][1]
            elif horizontal == "right":
                extremes_cartesian[emotion][vertical][horizontal][0][1] = extremes_cartesian[emotion]["higher"]["right"][0][1]

In [63]:
# Publish on TF
for emotion in extremes_cartesian:
    for vertical in extremes_cartesian[emotion]:
        for horizontal in extremes_cartesian[emotion][vertical]:
            trans = extremes_cartesian[emotion][vertical][horizontal][0]
            rot = extremes_cartesian[emotion][vertical][horizontal][1]
            tfb.sendTransform(trans, rot, rospy.Time.now(), "_".join([emotion, vertical, horizontal]), "world")

# Start here for later stick board update

# Compute sticks in cartesian space

In [53]:
# Base is ["lower"]["left"][0]
base_position = [-0.65, -0.68, 0.35]


In [60]:
# Starting from lower left (drawing direction)
# 
sticks = []
emotion = "hope"
y_interval = 0.015     # Interval of sticks in y direction
z_interval = 0.02      # Interval of sticks in z direction
group_interval = 0.02  # Interval between each group in y direction
num_groups = 7         # Number of stick groups per line
num_sticks = 4         # Number of sticks per group
height_stick = 0.04    # Length in z direction
num_height = 7         # Number of sticks in z direction (in height)

z_stick = base_position[2]
col = 0
for h in range(num_height):
    y_stick = base_position[1]
    for g in range(num_groups):
        for i in range(num_sticks):
            start_stick = [base_position[0], y_stick, z_stick]
            end_stick = [base_position[0], y_stick, height_stick + z_stick]
            sticks.append([start_stick, end_stick])

            tfb.sendTransform(sticks[-1][0], [0, 0, 0, 1], rospy.Time.now(), str(col), "world")
            tfb.sendTransform(sticks[-1][1], [0, 0, 0, 1], rospy.Time.now(), "e"+str(col), "world")
            y_stick += y_interval
            col += 1
        y_stick += group_interval
    z_stick += z_interval + height_stick
print(len(sticks), "sticks generated")

(196, 'sticks generated')


# Read approximate hand_to_stick transformation from tf

In [41]:
hope_lower_left = {u'right_j0': 1.611796875,
                   u'right_j1': -2.75679296875,
                   u'right_j2': -3.0216201171875,
                   u'right_j3': 1.296830078125,
                   u'right_j4': 1.161048828125,
                   u'right_j5': -1.98382421875,
                   u'right_j6': 1.0612880859375}

In [42]:
limb.move_to_joint_positions(hope_lower_left)
# Move to extreme lower left in JOINT space

In [72]:
hand_T_world = list(tfl.lookupTransform(world, effector, rospy.Time(0)))

In [74]:
hand_T_world

[[-0.5173524294713373, -0.6102119978073007, 0.2956745351668124],
 [0.5551387930838201,
  -0.04831044750848252,
  -0.8303267638763152,
  -0.006669802519999546]]

In [75]:
STICK_START = 0
STICK_END = 1
CONSIDERED_STICK = 4
stick_T_world = [sticks[CONSIDERED_STICK][STICK_START], [0, 0, 0, 1]]

In [76]:
stick_T_world

[[-0.65, -0.6, 0.35], [0, 0, 0, 1]]

In [77]:
hand_T_stick = transformations.multiply_transform(hand_T_world, transformations.inverse_transform(stick_T_world))

In [78]:
hand_T_stick

[[-0.4830532732485746, -1.265694376021281, -0.3929228942020103],
 [0.5551387930838201,
  -0.04831044750848252,
  -0.8303267638763152,
  -0.006669802519999546]]

# Get IK for each stick

In [57]:
for stick_id, stick in enumerate(sticks):
    hand_T_world = transformations.multiply_transform(hand_T_stick, [stick[STICK_START], [0,0,0,1]])
    tfb.sendTransform(hand_T_world[0], hand_T_world[1], rospy.Time.now(), "ik", "world")
    result = limb.ik_request(transformations.list_to_pose(hand_T_world))
    rospy.sleep(1)
    if result == False:
        raise RuntimeError("Can't compute stick " + str(stick_id))
        

[ERROR] [1554569328.118843]: INVALID POSE - No Valid Joint Solution Found.


RuntimeError: Can't compute stick 16

## Overwrite poses

In [65]:
hope = {"higher": {"left" : hope_higher_left, "right": hope_higher_right},
        "lower": {"left" : hope_lower_left, "right": hope_lower_right}}

In [66]:
fear = {}

In [67]:
extremes = {"hope": hope, "fear": fear}

In [68]:
with open("../config/poses.json", "w") as f:
    json.dump(extremes, f, indent=4)

# Kinematics

In [9]:
from sawyer.transformations import list_to_pose

In [10]:
from geometry_msgs.msg import Pose

In [16]:
limb.ik_request(list_to_pose([[0.452, 0.159, 0.216],  [0.707, 0.707, 0.002, 0.004]]))

[ERROR] [1554312105.446198]: INVALID POSE - No Valid Joint Solution Found.


False

In [15]:
limb.joint_angles()

{'right_j0': 0.0018564453125,
 'right_j1': -1.1825615234375,
 'right_j2': -0.0019326171875,
 'right_j3': 2.177814453125,
 'right_j4': 0.0020849609375,
 'right_j5': 0.5650791015625,
 'right_j6': 3.3153837890625}